In [1]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

In [2]:
lemmatizer = WordNetLemmatizer()

In [3]:
conversation = json.loads(open("conversation.json").read())

In [5]:
words = pickle.load(open("words.pkl","rb"))
classes = pickle.load(open("classes.pkl","rb"))
model = load_model("chatbot_model.h5")

In [6]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [7]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [8]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"ints": classes[r[0]], "probability": str(r[1])})
    return_list

In [10]:
def get_response(conversation_list, conversation_json):
    tag = conversation_list[0]['ints']
    list_of_conversation = conversation_json["conversation"]
    for i in list_of_conversation:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result

print("Chatbot is running. Ask her/him something!")

while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, conversation)
    print(res)

Chatbot is running. Ask her/him something!


TypeError: 'NoneType' object is not subscriptable